In [4]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import models
import metrics

%load_ext autoreload
%autoreload 2

In [5]:
def get_images(set_number, factor):
    images = []
    file_path = f"../Datasets/Test/Set{set_number}/image_SRF_{factor}/"
    for i in range(1, set_number + 1):
        hr_image = Image.open(f"{file_path}/img_{i:03d}_SRF_{factor}_HR.png")
        lr_image = Image.open(f"{file_path}/img_{i:03d}_SRF_{factor}_LR.png")
        images.append((hr_image, lr_image))
    return images

In [6]:
set_number = 5
factor = 2
images = get_images(set_number, factor)
print(images[0][0].size, images[0][1].size)

(512, 512) (256, 256)


In [7]:
lr_images = [img[1] for img in images]
hr_images = [img[0] for img in images]

In [8]:
def get_all_images():
    all_images = {5: {}, 14: {}}
    factors = [2, 3, 4]
    for set_number in [5, 14]:
        # all_images[set_number] = {}
        for factor in factors:
            images = get_images(set_number, factor)
            lr_images = [img[1] for img in images]
            hr_images = [img[0] for img in images]
            all_images[set_number][factor] = {
                'lr': lr_images,
                'hr': hr_images
            }
    return all_images

In [9]:
all_images = get_all_images()

In [10]:
print(all_images.keys())
print(all_images[5].keys())
print(all_images[5][2].keys())
print(len(all_images[5][2]['lr']), len(all_images[5][2]['hr']))

dict_keys([5, 14])
dict_keys([2, 3, 4])
dict_keys(['lr', 'hr'])
5 5


In [12]:
methods = ["bicubic", "nearest", "bilinear"]
#hacerlo en un pandas y msotra tabla final mas linda
for set_number in [5, 14]:
    for factor in [2]:
        print(f"##### Set {set_number}, Factor {factor}")
        for method in methods:
            print(f"Method: {method}")
            lr_images = all_images[set_number][factor]['lr']
            hr_images = all_images[set_number][factor]['hr']
            unsampled_images = models.upsample_images(lr_images, method=method, scale_factor=factor)
            psnr = np.mean([metrics.calculate_psnr(hr_images[i], unsampled_images[i]) for i in range(len(hr_images))])
            ssim = np.mean([metrics.calculate_ssim(hr_images[i], unsampled_images[i]) for i in range(len(hr_images))])
            print(f"PSNR: {psnr:.2f}, SSIM: {ssim:.4f}")
        print("\n")

##### Set 5, Factor 2
Method: bicubic
PSNR: 31.79, SSIM: 0.9178
Method: nearest
PSNR: 29.08, SSIM: 0.8913
Method: bilinear
PSNR: 30.42, SSIM: 0.8974


##### Set 14, Factor 2
Method: bicubic
PSNR: 28.31, SSIM: 0.8526
Method: nearest
PSNR: 26.72, SSIM: 0.8329
Method: bilinear
PSNR: 27.32, SSIM: 0.8223


